In [6]:
from GauAPIs.Molecule import Molecule
from GauAPIs import New_Calculator
from sys import argv
import numpy as np
import os


In [7]:
#define a temperate way to quickly test the feasibility of xtb API
filein = ''
fileout = 'test_res.out'

def get_external_coord(filein):
    with open(filein, 'r') as f:
        [atoms, deriva, charge ,spin] = [int(s) for s in f.readline().split()]
        ele = np.zeros((atoms,), dtype = int)
        
        atomlist = np.zeros((atoms,3), dtype = float)

        for i in range(atoms):
            axestr = f.readline().split()
            ele[i] = int(axestr[0])
            atomlist[i][0] = float(axestr[1])*0.52917724
            atomlist[i][1] = float(axestr[2])*0.52917724
            atomlist[i][2] = float(axestr[3])*0.52917724
    return ele, atomlist, deriva, charge, spin




In [ ]:

#print(ele, atomlist, deriva, charge, spin)
import numpy as np
elelist, coors, deriva, charge, spin = get_external_coord(filein)
print(deriva, charge, spin)
Mole = Molecule(elelist,coors,charge,spin,'testorc')
#Ene, Gradient = New_Calculator.calculate_mol_xTB(Mole,1)
#print(Ene)
#print(Gradient)


In [ ]:
try:
    from xtb.libxtb import VERBOSITY_FULL, VERBOSITY_MINIMAL, VERBOSITY_MUTED
    from xtb.interface import Environment, Calculator, Param
except ImportError:
    print('no xTB module found.')
env = Environment()
env.set_output("env_error.log")
env.set_verbosity(VERBOSITY_FULL)
if env.check != 0:
    env.show("Error message")
env.release_output()

os.system('export OMP_NUM_THREADS=4')
os.system('export MKL_NUM_THREADS=4')

In [ ]:
def calculate_mol_xTB(mol,xtbpara):
    #first solve the environment
    try:
        from xtb.libxtb import VERBOSITY_FULL, VERBOSITY_MINIMAL
        from xtb.interface import Environment, Calculator, Param
    except ImportError:
        print('no xTB module found.')
    
    env = Environment()
    env.set_output("env_error.log")
    env.set_verbosity(VERBOSITY_FULL)
    if env.check != 0:
        env.show("Error message")
    env.release_output()

    xtbpara = Param(xtbpara)
    #define structure data for molecule
    eles_nums = mol.elements
    coors = mol.coordinates
    coors = np.divide(coors,0.52917724)
    #define calculator based on input xtbpara and mol
    calc = Calculator(xtbpara,eles_nums,coors)
    calc.set_verbosity(VERBOSITY_MINIMAL)
    #Calculation
    res = calc.singlepoint()
    Ene = res.get_energy()
    Grad = res.get_gradient()
    print(Ene,Grad)
    return Ene, Grad

In [ ]:
calculate_mol_xTB(Mole,1)

In [ ]:

mol = Mole
#define structure data for molecule
eles_nums = mol.elements
coors = mol.coordinates
#print(eles_nums)
#print(coors)
#define calculator based on input xtbpara and mol

calc = Calculator(Param.GFN2xTB, eles_nums, coors)
calc.set_verbosity(VERBOSITY_MINIMAL)
calc.set_verbosity(VERBOSITY_MUTED) 
res = calc.singlepoint()
res.get_gradient()
#res.get_bond_orders()


In [ ]:
numbers = np.array([8, 1, 1])
positions = np.array([
[ 0.00000000000000, 0.00000000000000,-0.73578586109551],
[ 1.44183152868459, 0.00000000000000, 0.36789293054775],
[-1.44183152868459, 0.00000000000000, 0.36789293054775]])

print(numbers)
print(positions)
calc = Calculator(Param.GFN2xTB, numbers, positions)
calc.set_verbosity(VERBOSITY_MINIMAL)
res = calc.singlepoint()
print(res.get_gradient())
print('*'*10)
res.get_bond_orders()

In [ ]:

global eleslist
eleslist = ['H','He',
        'Li','Be','B','C','N','O','F','Ne',
        'Na','Mg','Al','Si','P','S','Cl','Ar',
        'K','Ca','Sc','Ti','V','Cr','Mn','Fe','Co','Ni','Cu','Zn','Ga','Ge','As','Se','Br','Kr']



In [ ]:
#mol2xyz(Mole)

nums = Mole.get_num_atoms()
print(Mole.elements)
fw = open('/share/home/Liaoyt/APIs/test.xyz','w')
fw.write('%d\n\n'%(nums))
for i in range(nums):
    print('1')
    fw.write('%s %f %f %f\n'%(eleslist[Mole.elements[i]-1],Mole.coordinates[i][0],Mole.coordinates[i][1],Mole.coordinates[i][2]))
fw.write('\n')
fw.close()        

In [ ]:
#ANI 

try:
    import torch
    import torchani
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

except ImportError:
    print('no ANI module found')



model = torchani.models.ANI2x(periodic_table_index=True).to(device).double()

In [ ]:

def calculate_mol_ani(mol,methodani):
    if methodani.lower() in ['ani-1x', 'ani-2x','ani-1ccx']:
        if methodani.lower() == 'ani-2x':
            model = torchani.models.ANI2x(periodic_table_index=True).to(device).double()
        elif methodani.lower() == 'ani-1ccx':
            model = torchani.models.ANI1ccx(periodic_table_index=True).to(device).double()
        elif methodani.lower() == 'ani-1x':
            model = torchani.models.ANI1x(periodic_table_index=True).to(device).double()
        else:
            print('Error: Method not recognised, Using ANI2x instead')
            model = torchani.models.ANI2x(periodic_table_index=True).to(device).double()
    
    print(Mole.elements)
    #have to convert from np to tensor 
    coordinates = torch.from_numpy(mol.coordinates).requires_grad_(True).unsqueeze(0)
    species = torch.from_numpy(mol.elements).unsqueeze(0)
    ene = model((species, coordinates)).energies
    derivative = torch.autograd.grad(ene.sum(), coordinates)[0]
    force = -derivative
    #convert tensor to numpy and return
    return ene.item(),derivative.numpy()


In [ ]:
a,b = calculate_mol_ani(Mole,'ani-2x')


In [ ]:
#aiqm test
import mlatom as ml
import os

def mol2xyz(mol):
    nums = mol.get_num_atoms()
    print(mol.elements)
    fw = open('/share/home/Liaoyt/APIs/test.xyz','w')
    fw.write('%d\n\n'%(nums))
    for i in range(nums):
        fw.write('%s %f %f %f\n'%(eleslist[mol.elements[i]-1],mol.coordinates[i][0],mol.coordinates[i][1],mol.coordinates[i][2]))
    fw.write('\n')
    fw.close()        


In [ ]:

def calculate_mol_mlatom(xyzfile,charge,spin,methodmlatm):
    #read molecule in 
    mol = ml.data.molecule.from_xyz_file(xyzfile)
    mol.charge = charge
    mol.multiplicity = spin
    #define model
    model = ml.models.methods(method=methodmlatm)
    
    #calculate energy and force
    model.predict(molecule=mol)

    ene = mol.energy
    grad = mol.get_energy_gradients()
    return ene,grad


#os.system("export mndobin=/share/apps/MNDO/BIN/mndo")
#os.system('export dftd4bin=/share/home/Liaoyt/app/dftd4-3.5.0/bin/dftd4')


In [2]:
#os.system('source /share/home/Liaoyt/bin/env/aiqm1-env.sh')
import mlatom as ml
xyz = '/share/home/Liaoyt/apiTEst/AIQM1-input.xyz'
mol = ml.data.molecule.from_xyz_file(xyz)
charge=-1
spin=1
model = ml.models.methods(method='AIQM1')
model.predict(molecule=mol)



There is no gradient information in fort.15. Please check your input!


In [ ]:
def Get_orca_grad(outname,mol):
    fg = open(outname,"r")
    lines = fg.readlines()
    fg.close()
    nums = mol.get_num_atoms()
    print(nums)
    grad_arr = np.zeros((nums,3),dtype=float)
    for line in lines:  
        if "CARTESIAN GRADIENT" in line:
            index_of_grad = lines.index(line)
            break
        counts = 0
    for grads in lines[index_of_grad+3:index_of_grad+2+nums]:
        print(counts)
        grad_arr[counts] = np.array([float(grads.split()[3]),float(grads.split()[4]),float(grads.split()[5])])
        print(grad_arr)
        counts += 1
    
    return grad_arr

In [ ]:
grad_arr = np.zeros((6,3),dtype=float)
for i in range(len(grad_arr)):
    grad_arr[i] = np.array([1,1,1])
    print(grad_arr)

In [ ]:
test_grad = Get_orca_grad('/share/home/Liaoyt/APIs/testorc.log',Mole)

In [9]:
import resource
import psutil
b2a = 0.52917724

def calculate_mol_tblite(mol,xtbpara,ncpu=1):
    try:
        from tblite.interface import Calculator
    except ImportError:
        print('no tblite module found.')
    if ncpu != 1:
    #set parallelization and stacksize
        os.environ['OMP_STACKSIZE'] = f'{ncpu}G'
        os.environ['OMP_NUM_THREADS'] = f'{len(psutil.Process().cpu_affinity())},1'
        os.environ['OMP_MAX_ACTIVE_LEVELS'] = '1'
    resource.setrlimit(resource.RLIMIT_STACK, (resource.RLIM_INFINITY, resource.RLIM_INFINITY))
    #calculation
    numbers = mol.elements
    coordinates = np.divide(mol.coordinates,b2a)
    calc = Calculator(xtbpara,numbers,coordinates)
    resu = calc.singlepoint()
    Ene = resu.get("energy")
    Grad = resu.get("gradient")
    return Ene, Grad
    

In [11]:
INfile = '/share/home/Liaoyt/Work/DPI/1br5/3_layer/opt/test/Gau-9473.EIn'
ele, atomlist, deriva, charge, spin = get_external_coord(INfile)

mole = Molecule(ele,atomlist,charge,spin)

Ene, gra = calculate_mol_tblite(mole,'GFN2-xTB')
print(Ene)
print(gra)


------------------------------------------------------------
  cycle        total energy    energy error   density error
------------------------------------------------------------
      1     -135.6870823273  -1.3734498E+02   2.2325584E-01
      2     -136.5762632621  -8.8918093E-01   1.0332834E-01
      3     -136.4786296303   9.7633632E-02   8.5630428E-02
      4     -135.8153819578   6.6324767E-01   1.0110887E-01
      5     -137.2371046882  -1.4217227E+00   5.2002792E-02
      6     -137.1742374689   6.2867219E-02   4.1386634E-02
      7     -137.3613507554  -1.8711329E-01   2.6791846E-02
      8     -137.3094704880   5.1880267E-02   2.3485446E-02
      9     -137.3767683047  -6.7297817E-02   1.7235996E-02
     10     -137.3865674794  -9.7991747E-03   1.2004160E-02
     11     -137.3796739087   6.8935707E-03   1.3488844E-02
     12     -137.3865720925  -6.8981838E-03   1.0054014E-02
     13     -137.3227749465   6.3797146E-02   2.0307952E-02
     14     -137.3927741996  -6.999925